In [1]:
%matplotlib inline
from keras.models import Model
from keras.layers import *
from keras.optimizers import *
from keras.utils import plot_model

import tensorflow as tf
import keras.initializers as kinit
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from dqnroute.utils import *

from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
data = pd.read_csv('logs/data_generated2.csv', names=get_data_cols(10))

In [3]:
import gc
gc.collect()

0

In [4]:
data.head()

,time,pkg_id,cur_node,dst_0,dst_1,dst_2,dst_3,dst_4,dst_5,dst_6,...,predict_0,predict_1,predict_2,predict_3,predict_4,predict_5,predict_6,predict_7,predict_8,predict_9
0,10.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-10.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-30.0,-1000000.0,-30.0,-1000000.0
1,20.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1000000.0,-20.0,-20.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0
2,20.0,2.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,-20.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-20.0,-1000000.0,-1000000.0,-1000000.0
3,30.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,-1000000.0,-10.0,-30.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0
4,40.0,2.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,-20.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-20.0,-1000000.0,-20.0,-1000000.0


In [5]:
target_cols = get_target_cols(10)
neighbors_cols = get_neighbors_cols(10)
addr_cols = get_addr_cols(10)
dst_cols = get_dst_cols(10)
amatrix_cols = get_amatrix_cols(10)

targets = data[target_cols]
neighbors_inp = data[neighbors_cols]
addr_inp = data[addr_cols]
dst_inp = data[dst_cols]
amatrix_inp = data[amatrix_cols]

In [6]:
neighbors_inp.head()

,neighbors_0,neighbors_1,neighbors_2,neighbors_3,neighbors_4,neighbors_5,neighbors_6,neighbors_7,neighbors_8,neighbors_9
0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [7]:
targets.head()

,predict_0,predict_1,predict_2,predict_3,predict_4,predict_5,predict_6,predict_7,predict_8,predict_9
0,-10.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-30.0,-1000000.0,-30.0,-1000000.0
1,-1000000.0,-20.0,-20.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0
2,-20.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-20.0,-1000000.0,-1000000.0,-1000000.0
3,-1000000.0,-10.0,-30.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0
4,-20.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-20.0,-1000000.0,-20.0,-1000000.0


In [8]:
neighbors_input_dim = neighbors_inp.shape[1]
addr_input_dim = addr_inp.shape[1]
dst_input_dim = dst_inp.shape[1]
amatrix_input_dim = amatrix_inp.shape[1]
output_dim = targets.shape[1]

In [13]:
neighbors_input = Input(shape=(neighbors_input_dim,), name='neighbors_input')
addr_input = Input(shape=(addr_input_dim,), name='addr_input')
dst_input = Input(shape=(dst_input_dim,), name='dst_input')
amatrix_input = Input(shape=(amatrix_input_dim,), name='amatrix_input')

x = concatenate([neighbors_input, addr_input, dst_input, amatrix_input])
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)
x = Dense(output_dim, activation='linear')(x)

y = Lambda(lambda x: (1 - x) * -1000000)(neighbors_input)

x = add([x, y])
model = Model(inputs=[neighbors_input, addr_input, dst_input, amatrix_input], outputs=x)

model.compile(loss='mse', optimizer='rmsprop')

In [14]:
N_train, N_test, A_train, A_test, D_train, D_test, M_train, M_test, y_train, y_test = \
    train_test_split(neighbors_inp.values,
                     addr_inp.values,
                     dst_inp.values,
                     amatrix_inp.values,
                     targets.values,
                     test_size=0.2, random_state=42)

In [12]:
model.fit([N_train, A_train, D_train, M_train], y_train, epochs=5)

Epoch 1/5
184857/184857 [==============================] - 9s - loss: 21.5205     
Epoch 2/5
184857/184857 [==============================] - 9s - loss: 5.2272     
Epoch 3/5
184857/184857 [==============================] - 9s - loss: 4.0865     
Epoch 4/5
184857/184857 [==============================] - 10s - loss: 3.2881    
Epoch 5/5
184857/184857 [==============================] - 10s - loss: 2.7673    


In [13]:
model.evaluate([N_test, A_test, D_test, M_test], y_test)

44608/46215 [===========================>..] - ETA: 0s

2.4100291145463069

In [ ]:
model.predict(list(map(lambda x: x.reshape((1, 10)), [N_test[0], A_test[0], D_test[0]])) + [M_test[0].reshape((1, 100))])

In [10]:
y_test[0]

array([ -1.00000000e+06,  -1.00000000e+06,  -2.00000000e+01,
        -1.00000000e+06,  -1.00000000e+06,  -1.00000000e+06,
        -1.00000000e+06,  -1.00000000e+06,  -1.00000000e+06,
        -1.00000000e+06])

In [ ]:
model.save('../router_network.h5')

In [ ]:
plot_model(model, show_shapes=True)

In [12]:
data.shape

(231072, 143)

In [9]:
from dqnroute.q_network import Qnetwork

In [10]:
def _make_batches(size, batch_size):
    num_batches = int(np.ceil(size / float(batch_size)))
    return [(i * batch_size, min(size, (i + 1) * batch_size))
            for i in range(0, num_batches)]

def fit_model(model, x, y, batch_size=32, epochs=1):
    for i in range(epochs):
        print('Epoch {}...'.format(i))
        Ns, As, Ds, Ms = x
        count = Ns.shape[0]
        
        sum_loss = 0
        loss_cnt = 0
        for (a, b) in _make_batches(count, batch_size):
            Nb = Ns[a:b]
            Ab = As[a:b]
            Db = Ds[a:b]
            Mb = Ms[a:b]
            target_b = y[a:b]
            loss, _ = sess.run([model.loss, model.updateModel],
                               feed_dict={model.neighbors_input:Nb, model.addr_input:Ab,
                                          model.dst_input:Db, model.amatrix_input:Mb,
                                          model.target:target_b})
            sum_loss += loss
            loss_cnt += 1.0
        print('loss: {}'.format(sum_loss / loss_cnt))

In [20]:
tf.reset_default_graph()
rnn_cell = tf.contrib.rnn.BasicLSTMCell(num_units=64)
tf_model = Qnetwork(10, rnn_cell, 'mda')
init = tf.global_variables_initializer()
save_path = '../tf_model'
saver = tf.train.Saver()

sess = tf.Session()
sess.run(init)

In [23]:
fit_model(tf_model, [N_train, A_train, D_train, M_train], y_train, epochs=10)

Epoch 0...
loss: 0.3293299322010572
Epoch 1...
loss: 0.3259439491185635
Epoch 2...
loss: 0.32216161352520856
Epoch 3...
loss: 0.31976644406428856
Epoch 4...
loss: 0.31676842226626367
Epoch 5...
loss: 0.3133126296346753
Epoch 6...
loss: 0.3102817749554607
Epoch 7...
loss: 0.30748034813385716
Epoch 8...
loss: 0.3050080202229712
Epoch 9...
loss: 0.301901682353678


In [24]:
saver.save(sess, save_path)

'../tf_model'

In [26]:
N0, A0, D0, M0 = list(map(lambda x: x.reshape((1, 10)), [N_test[0], A_test[0], D_test[0]])) + [M_test[0].reshape((1, 100))]

In [31]:
sess.run(tf_model.Qout,
         feed_dict={tf_model.neighbors_input:N0, tf_model.addr_input:A0,
                    tf_model.dst_input:D0, tf_model.amatrix_input:M0})

array([[ -9.99999938e+05,  -1.00000000e+06,  -1.94056034e+01,
         -1.00000006e+06,  -9.99999938e+05,  -9.99999938e+05,
         -9.99999750e+05,  -9.99999875e+05,  -1.00000000e+06,
         -1.00000031e+06]], dtype=float32)

array([ -1.00000000e+06,  -1.00000000e+06,  -2.00000000e+01,
        -1.00000000e+06,  -1.00000000e+06,  -1.00000000e+06,
        -1.00000000e+06,  -1.00000000e+06,  -1.00000000e+06,
        -1.00000000e+06])

In [35]:
save_dir = '../tf_model'
saver.save(sess, save_dir + '/model1.ckpt')

'../tf_model/model1.ckpt'